In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor
train_data = TabularDataset('X_sample.csv')
train_data.head()

,0,1,2,3,4,5,6,level
0,13221.324850,11557.585690,8915.761951,6346.741284,4068.865074,2080.802654,424.587129,5
1,13513.510820,11849.516620,9208.116046,6636.700489,4355.318842,2367.796144,652.028388,6
2,9738.352333,8074.327426,5432.637589,2861.404817,580.598189,1410.785921,3128.623583,3
3,13223.178290,11559.348250,8917.436429,6347.867710,4069.076181,2079.819210,406.122921,5
4,10728.853470,9064.832279,6423.220369,3851.901161,1570.725699,424.306768,2138.315602,5


In [2]:
label = 'level'
print("Summary of class variable: \n", train_data[label].describe())

Summary of class variable: 
 count    84044.000000
mean         3.499726
std          1.709293
min          1.000000
25%          2.000000
50%          3.000000
75%          5.000000
max          6.000000
Name: level, dtype: float64


/Users/han/opt/anaconda3/envs/Dissertation/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
save_path = 'agModels-predictClass'  # specifies folder to store trained models
predictor = TabularPredictor(label=label, path=save_path).fit(train_data)

Beginning AutoGluon training ...
AutoGluon will save models to "agModels-predictClass/"
AutoGluon Version:  0.2.0
Train Data Rows:    84044
Train Data Columns: 7
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == int, but few unique label-values observed).
	6 unique label values:  [5, 6, 3, 1, 2, 4]
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Train Data Class Count: 6
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    6310.81 MB
	Train Data (Original)  Memory Usage: 4.71 MB (0.1% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 

In [6]:
test_data = TabularDataset('X_sample_2015.csv')
y_test = test_data[label]  # values to predict
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

/Users/han/opt/anaconda3/envs/Dissertation/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Loaded data from: X_sample_2015.csv | Columns = 8 / 8 | Rows = 172567 -> 172567


,0,1,2,3,4,5,6
0,1754.316486,1166.820125,610.159171,47.170226,924.643236,2903.587747,6229.777641
1,1549.843744,962.473935,405.715853,244.598137,1128.598801,3107.595242,6433.882168
2,1584.161613,996.850026,440.004049,210.312865,1094.199869,3073.169024,6399.469848
3,904.718901,317.655110,241.610945,889.781272,1774.230745,3753.254645,7079.517069
4,1974.140960,1387.312340,830.549963,185.543034,705.135026,2683.124638,6009.406128


In [7]:
predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict(test_data_nolab)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

/Users/han/opt/anaconda3/envs/Dissertation/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
sklearn.neighbors.KNeighborsMixin.kneighbors: uses Intel(R) oneAPI Data Analytics Library solver
sklearn.neighbors.KNeighborsMixin.kneighbors: uses Intel(R) oneAPI Data Analytics Library solver
Evaluation: accuracy on test data: 0.02427462956416928
Evaluations on test data:
{
    "accuracy": 0.02427462956416928,
    "balanced_accuracy": 0.1824501212522046,
    "mcc": -0.12360757710046237
}


Predictions:  
 0         3
1         3
2         3
3         2
4         2
         ..
172562    2
172563    2
172564    2
172565    6
172566    3
Name: level, Length: 172567, dtype: int64


In [8]:
predictor.leaderboard(test_data, silent=True)

/Users/han/opt/anaconda3/envs/Dissertation/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
sklearn.neighbors.KNeighborsMixin.kneighbors: uses Intel(R) oneAPI Data Analytics Library solver
sklearn.neighbors.KNeighborsMixin.kneighbors: uses Intel(R) oneAPI Data Analytics Library solver


,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMLarge,0.322895,0.8124,0.269908,0.006589,5.656194,0.269908,0.006589,5.656194,1,True,12
1,XGBoost,0.118609,0.8568,1.529160,0.020840,62.562384,1.529160,0.020840,62.562384,1,True,11
2,KNeighborsUnif,0.072575,0.8636,5.245139,0.033791,6.552634,5.245139,0.033791,6.552634,1,True,1
3,KNeighborsDist,0.071566,0.8876,5.582812,0.047352,0.122992,5.582812,0.047352,0.122992,1,True,2
4,RandomForestGini,0.058968,0.8948,1.998447,0.169282,20.989477,1.998447,0.169282,20.989477,1,True,6
5,RandomForestEntr,0.052513,0.8948,1.680548,0.160952,29.294059,1.680548,0.160952,29.294059,1,True,7
6,ExtraTreesGini,0.041653,0.9000,2.396617,0.152012,4.947451,2.396617,0.152012,4.947451,1,True,9
7,ExtraTreesEntr,0.035447,0.9016,2.202719,0.127123,4.723023,2.202719,0.127123,4.723023,1,True,10
8,LightGBM,0.024373,0.8280,6.136643,0.101953,14.277996,6.136643,0.101953,14.277996,1,True,5
9,WeightedEnsemble_L2,0.024275,0.9036,18.391612,0.437101,199.473324,0.162254,0.001928,0.863257,2,True,13
